# Méthodes avancées pour la factorisation généraliste
Pour la factorisation ou le test de primalité des nombres de Mersenne ou de Fermat, par exemple, on utilise en réalité des méthodes spécifiques issues de théorème liés à la nature des nombres.

On parle de méthode généraliste pour des nombres entiers dont on ne connait que la valeur et aucune propriété, comme avec un nombre choisi au hasard dans un intervalle.

Pour aborder les méthodes de factorisation,
- on peut d'abord voir une version améliorée du crible d'Ératosthène,
- il faut une version améliorée des tests de primalité,

Ce carnet est clairement pour le post-bac, avec des parties plus ou moins abordables.

---
## Crible d'Ératosthène optimisé en Python
On utilise les `bytearray` plutôt que les listes d'entiers pour un gain de place, et donc de temps conséquent.
On ne travaille aussi qu'avec les entiers impairs. On peut sans difficulté faire le crible jusqu'à $10^7$ sur tablette.

Pour une version encore plus optimisée, on utilisera un crible segmenté ; on fera le crible par tranches, et les données de taille plus restreintes resteront en mémoire cache bien plus rapide. On pourra alors faire un crible, par tranches, jusqu'à $10^9$ en quelques secondes, avec peu de mémoire utilisée.

In [1]:
from itertools import compress, islice

def eratosthene_impair(borne):
    """
    Retourne le crible des nombres impairs premiers inférieurs à borne.
    crible[n] vaut True ou False, suivant que 2*n+1 est premier ou non.
    """
    BA = bytearray
    n = (borne - 1) // 2
    crible = BA([1])*(n+1)
    # crible[i] indiquera la primalité de 2i + 1
    crible[0] = 0
    for i in range((int(borne**0.5)+1)//2):
        if crible[i]:
            p = 2*i + 1 # p est premier
            i2 = i * (i+1) << 1
            crible[i2::p] = BA(1 + (n-i2)//p)
    return crible

borne = 10**7
crible = eratosthene_impair(borne)
premier = [2]
premier.extend(compress(range(1,borne,2), crible))
print(premier[:20])

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71]


### Exercice : Crible d'Ératosthène segmenté
Étudier la version précédente et construire un crible segmenté :
- À partir d'un crible classique qui donne les nombres premiers inférieurs à $M$,
- construire un crible qui donne la primalité pour tous les entiers dans un intervalle $[a ; b]$, avec $a < b < M^2$.

Tester votre code *Python* [sur ce problème en ligne](https://www.spoj.com/problems/PRIME1/), ou sur [celui-là un peu plus difficile](https://www.spoj.com/problems/PRINT/).
> - Pour PRIME1, pouvez-vous faire mieux que 0.03s en *Python3* ?
> - Pour PRINT,  pouvez-vous faire mieux que 0.62s en *Python3* ?

---
## Tests de primalité probabilistes
Nous ne donnerons pas ici les démonstrations des théorèmes ; nous nous concentrons sur leur utilisation.

### [Test de primalité de Fermat](https://fr.wikipedia.org/wiki/Test_de_primalit%C3%A9_de_Fermat)
Théorème :
> Si $p$ est premier, alors pour tout $a$ tel que $1<a<p$, on a $a^{p-1}\equiv 1 \mod[p]$

La réciproque est fausse, mais on utilise la contraposée pour avoir un test probabiliste de primalité. Test que l'on peut rendre déterministe sous certaines conditions.

In [2]:
from random import randint

In [3]:
def est_pseudo_premier_v1(n):
    """
    Retourne True quand n est premier, mais parfois aussi quand n est composé.
    Ne retourne jamais False quand n est premier.
    """
    nb_tour_boucle = 5
    if n < 2: return False
    if n < 4: return True
    if (n % 2 == 0) or (n % 3 == 0): return False
    for _ in range(nb_tour_boucle):
        a = randint(2, n-1)
        if n % a == 0: return False
        if pow(a, n-1, n) != 1: return False
    return True

for n in range(3, 10**6, 2):
    if crible[n//2]: assert est_pseudo_premier_v1(n)
    if est_pseudo_premier_v1(n) != crible[n//2]:
        print(f"Oups, avec {n}")

Oups, avec 6601
Oups, avec 29341
Oups, avec 63973
Oups, avec 162401
Oups, avec 252601
Oups, avec 294409
Oups, avec 334153
Oups, avec 399001
Oups, avec 488881
Oups, avec 512461
Oups, avec 530881
Oups, avec 552721
Oups, avec 670033


On constate que pour `nb_tour_boucle = 5`, la quantité de nombres qui échouent le test est de l'ordre de la quinzaine, nombres parmi les entiers jusqu'à $10^6$.
- Essayer de lancer plusieurs fois le code, la liste varie ; le test est probabiliste.
- Essayer avec d'autres valeurs de `nb_tour_boucle`.

#### Variante déterministe
Le test de primalité de Fermat a une complexité en $\Theta(\log n)$ ce qui est bien meilleur que notre précédent test individuel en $\mathcal O\left(\sqrt n\right)$. Il est très intéressant, mais parfois faux !

On peut rendre ce test déterministe sur un intervalle. On va le modifier un peu.
- On commence par tester la divisibilité par de petits nombres premiers. C'est rapide et efficace pour les entiers qui sont peu lisses.
- On fixe judicieusement les témoins du tests Fermat en fonction de l'intervalle de travail.
- On choisira une meilleure méthode pour des entiers plus grands...

In [4]:
def est_premier_vD1(n):
    """
    Retourne la primalité de n
    correct pour n < 9_006_401
    """
    if n < 2: return False
    if n < 4: return True
    if (n % 2 == 0) or (n % 3 == 0): return False
    if n < 5*5: return True # les composés inférieurs à 25 ont un facteur premier inférieur à 5, donc 2 ou 3, et ont déjà été traités
    for p in [5, 7, 11, 13, 17, 19, 23]:
        if n % p == 0: return False
    if n < 29*29: return True # même remarque ; jusqu'à 841 !
    for p in [29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269]: #, 271, ...
        if n % p == 0: return False
    if n < 271*271: return True # même remarque jusqu'à 73441 !
    # On commence le test de Fermat avec un unique témoin égal à 2, puis 2 et 3
    if pow(2, n-1, n) != 1: return False
    if n < 219781: return True
    if pow(3, n-1, n) != 1: return False
    if n < 226801: return True
    if pow(5, n-1, n) != 1: return False
    if n < 721801: return True
    if pow(13, n-1, n) != 1: return False
    if n < 9006401: return True
    # n'est plus déterministe après ça !!!
    return True

for n in range(1, 10**7, 2):
    assert est_premier_vD1(n) == crible[n//2], f"Échec avec n = {n}"

AssertionError: Échec avec n = 9006401

### [Test de primalité de Miller-Rabin](https://fr.wikipedia.org/wiki/Test_de_primalit%C3%A9_de_Miller-Rabin)
Le test de Miller-Rabin étend le test de Fermat : la propriété est un raffinement du petit théorème de Fermat.

Avec $n>2$ que l'on écrit $1+2^sd$, où $d$ est impair et $s$ non nul :

> Si $a^d \not\equiv 1 \pmod{n}$ et $\forall r \in \{0, 1, \cdots, s-1\} a^{2^rd}\not\equiv -1 \pmod{n}$
>
> alors $n$ est composé, et $a$ est appelé un *témoin de Miller* pour le fait que $n$ est composé.

La réciproque est fausse, mais contrairement au test de Fermat il n'y a pas l'équivalent des nombres de Carmichael, et de plus $\frac34$ au moins des entiers $a$ ($1<a<n$) sont des témoins de Miller pour $n$. Ce qui conduit à l'algorithme probabiliste :
- répéter $k$ fois le test de Miller-Rabin
    - si un témoin de Miller est positif, on peut déclarer $n$ composé.
    - Si aucun test parmi les $e$ tests n'est positif, on déclare $n$ pseudo-premier, avec une probabilité d'erreur inférieure à $\left(\frac14\right)^k$ sur le fait que $n$ soit premier.
    
Ci-dessous un code non optimisé, mais avec une écriture fonctionnelle.

In [6]:
from random import sample

def miller_rabin(n, nb_test = 3):
    #assert n-1 > nb_test
    #assert n % 2 == 1
    s = 0
    d = n - 1
    while not(d & 1):
        d >>= 1
        s += 1
    #assert n == 1 + d*2**s
    return not any((pow(a, d, n) != 1) and all(pow(a, d << r, n) != n-1 for r in range(s)) for a in sample(range(2, n), nb_test))

for n in range(3, 10**7, 2):
    for p in premier[:20]:
        if n % p == 0: break
    else:
        # arrivé ici, n est premier[19]-lisse
        if miller_rabin(n) != crible[n//2]:
            print(f"Oups avec n = {n}")

Oups avec n = 1433407
Oups avec n = 1869211


C'est un peu long pour tester tous les $n < 10^7$, mais on constate ici,
que quelques rares valeurs échouent au test ; parfois aucune, c'est aléatoire.

Pour diminuer la probabilité d'erreur, on augmente `nb_test` qui était ici égal à 3. On peut aussi bien sûr augmenter les essais de divisions par des nombres premiers ; il faut trouver le bon compromis vitesse taux d'erreur.

Pour $n$ plus grand il suffit d'augmenter `nb_test` pour avoir une quasi-certitude sur la primalité de $n$. Ce n'est pas satisfaisant dans certaines situations, où on veut une certitude absolue. On veut rendre le test déterministe.

#### [Versions déterministes](https://fr.wikipedia.org/wiki/Test_de_primalit%C3%A9_de_Miller-Rabin#Versions_d%C3%A9terministes)
- Si $n < 2~047$, il suffit de tester le témoin $a = 2$;
- Si $n < 1~373~653$, il suffit de tester les témoins $a$ dans $(2, 3)$ ;
- Si $n < 9~080~191$, il suffit de tester les témoins $a$ dans $(31 ,73)$ ;
- Si $n < 25~326~001$, il suffit de tester les témoins $a$ dans $(2, 3, ,5)$ ;
- Si $n < 3~215~031~751$, il suffit de tester les témoins $a$ dans $(2, 3, 5, ,7)$ ;
- Si $n < 4~759~123~141$, il suffit de tester les témoins $a$ dans $(2, 7, ,61)$ ;
- Si $n < 1~122~004~669~633$, il suffit de tester les témoins $a$ dans $(2, 13, 23, ,1662803)$ ;
- Si $n < 2~152~302~898~747$, il suffit de tester les témoins $a$ dans $(2, 3, 5, 7, ,11)$ ;
- Si $n < 3~474~749~660~383$, il suffit de tester les témoins $a$ dans $(2, 3, 5, 7, 11, ,13)$ ;
- Si $n < 341~550~071~728~321$, il suffit de tester les témoins $a$ dans $(2, 3, 5, 7, 11, 13, ,17)$.

On peut continuer ainsi et trouver la limite de validité du test avec comme témoins les premiers nombres premiers.

On peut aussi conjecturer une heuristique sur le nombre de témoins premiers qui assure une version déterministe.

Il a été démontré qu'il suffit de tester tous les témoins dans l'intervalle $[2, \textrm{min}(n−2, ⌊2(\log n)^2⌋)]$, cependant cette preuve dépend de la validité de l'hypothèse de Riemman généralisé (GRH). Un meilleure borne existe sous l'hypothèse ERH.

> Il existe d'autres tests de primalité déterministes qui ne dépendent pas de l'hypothèse de Riemann et qui sont plus rapides.
>  - Pour les entiers sur 64-bits [le test Baillie–PSW](https://en.wikipedia.org/wiki/Baillie%E2%80%93PSW_primality_test) est déterministe et bien plus rapide.
>  - [Le test Adleman–Pomerance–Rumely](https://en.wikipedia.org/wiki/Adleman%E2%80%93Pomerance%E2%80%93Rumely_primality_test)
>  - [Le test avec les courbes elliptiques](https://en.wikipedia.org/wiki/Elliptic_curve_primality)